In [27]:
from google.cloud import bigquery
import tensorflow as tf

import pandas as pd

import json
import os
from datetime import date,datetime,timedelta

import functions_framework

from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest

print(tf.__version__)

2.11.1


In [28]:
# @functions_framework.http
# def predict_incident_severity_by_tf(request):

In [29]:
PROJECT_ID='pongthorn'
dataset_id='DemoSMartDW'
#PATH_FOLDER_ARTIFACTS="gs://demo-tuned-tf-incident-pongthorn/model_tuned"
PATH_FOLDER_ARTIFACTS="model" 

predict_from_date=os.environ.get('predict_from_date', '')
all_prediction=os.environ.get('all_prediction', '0')  # 1 is all , 0 is 1 day

print(f"Prediction From = {predict_from_date}")
print(f"All prediction = {all_prediction}")

# predict_from_date='2023-03-01'

# map_sevirity_to_class={'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}

Prediction From = 
All prediction = 0


In [14]:
table_id = f"{PROJECT_ID}.{dataset_id}.new_incident"
predictResult_table_id=f"{PROJECT_ID}.{dataset_id}.new_result_prediction_incident"
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [15]:
mapping_file="incident_severity_to_class.json"
with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)

print(map_sevirity_to_class)

{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [16]:
# Get today's date
prediction_datetime=datetime.now()

today = date.today()

# Yesterday date
if predict_from_date=='':
 yesterday = today - timedelta(days = 1)
 str_yesterday=yesterday.strftime('%Y-%m-%d')
else:
 str_yesterday=predict_from_date

str_today=today.strftime('%Y-%m-%d')

print(f"Get data between {str_yesterday} to {str_today} to predict sevirity level")

Get data between 2023-04-19 to 2023-04-20 to predict sevirity level


In [17]:
client = bigquery.Client(PROJECT_ID)
def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 return df

In [18]:
if int(all_prediction)==0:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     WHERE DATE(imported_at) >= '{str_yesterday}' and DATE(imported_at) < '{str_today}' 
     order by imported_at
    """
else:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     order by imported_at
    """

print(sql)


    SELECT *  FROM `pongthorn.DemoSMartDW.new_incident` 
     order by imported_at
    


In [19]:

#LIMIT 2
dfNewData=load_data_bq(sql)
dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='last')

dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)


print(dfNewData.info())
# print(dfNewData)

if len(dfNewData)==0:
    print("No Data To predict")
    quit()
    #return "No Data To predict"
    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 47
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         48 non-null     Int64         
 1   severity_id                48 non-null     Int64         
 2   severity                   48 non-null     int64         
 3   severity_name              48 non-null     object        
 4   sla                        48 non-null     object        
 5   product_type               48 non-null     object        
 6   brand                      48 non-null     object        
 7   service_type               48 non-null     object        
 8   incident_type              48 non-null     object        
 9   open_to_close_hour         48 non-null     float64       
 10  response_to_resolved_hour  48 non-null     float64       
 11  imported_at                48 non-null     datetime64[ns]
dtypes: Int64(2

In [20]:
try:
    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # print(model.summary())
except Exception as error:
    
  print(str(error))
  raise error


Load from model


In [21]:
pdPrediction=pd.DataFrame(columns=['_id','predict_severity','prob_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


      predictionResult = model.predict(input_dict)
      result_str=','.join([ str(prob) for prob in predictionResult[0]])  
      print(result_str)   

      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      
      dictPrediction={'_id':incident_id, 'predict_severity':_class,'prob_severity':result_str} 
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([dictPrediction])] )

      print(f"{prob_pct} %   as {_class}")     
      print("======================================================================================")
            
dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData['predict_severity']=dfPredictData['predict_severity'].astype('int')
dfPredictData=dfPredictData[['id','prob_severity','predict_severity','severity']]
dfPredictData['prediction_item_date']= datetime.strptime(str_yesterday, '%Y-%m-%d')
dfPredictData['prediction_datetime']=prediction_datetime

2586 - 3(Critical)
1/1 [==============================] - 1s 893ms/step
0.0059843385,0.07343206,0.68859875,0.23198481
[[18.8594   20.175303 37.323605 23.641697]] %   as 2
2640 - 2(Major)
1/1 [==============================] - 0s 87ms/step
0.008027394,0.06385942,0.8865954,0.041517723
[[18.185366 19.229568 43.780354 18.80471 ]] %   as 2
2613 - 2(Major)
1/1 [==============================] - 0s 80ms/step
0.011942738,0.17643033,0.51847214,0.2931548
[[19.368416 22.831274 32.142303 25.658009]] %   as 2
2601 - 2(Major)
1/1 [==============================] - 0s 93ms/step
0.011335231,0.07954422,0.88664037,0.022480229
[[18.2435   19.531294 43.77724  18.447962]] %   as 2
2589 - 2(Major)
1/1 [==============================] - 0s 85ms/step
0.02882676,0.28550455,0.6352314,0.050437268
[[19.423885 25.10787  35.62003  19.848213]] %   as 2
2632 - 2(Major)
1/1 [==============================] - 0s 85ms/step
0.011307364,0.19144018,0.69342905,0.103823476
[[18.966776 22.710367 37.517612 20.805243]] %   as 2

In [22]:
print(dfPredictData.info())
print(dfPredictData)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 47
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    48 non-null     object        
 1   prob_severity         48 non-null     object        
 2   predict_severity      48 non-null     int32         
 3   severity              48 non-null     int64         
 4   prediction_item_date  48 non-null     datetime64[ns]
 5   prediction_datetime   48 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(1), object(2)
memory usage: 2.4+ KB
None
      id                                     prob_severity  predict_severity  \
0   2586     0.0059843385,0.07343206,0.68859875,0.23198481                 2   
1   2640      0.008027394,0.06385942,0.8865954,0.041517723                 2   
2   2613       0.011942738,0.17643033,0.51847214,0.2931548                 2   
3   2601     0.011335231,0.079

In [23]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prob_severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("predict_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity", "INTEGER", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_item_date", "DATETIME", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.DemoSMartDW.new_result_prediction_incident already exists.


In [24]:
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            dfPredictData, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(dfPredictData), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  48 Imported bigquery successfully


In [ ]:
# return 'ok'

In [ ]:
# if __name__ == "__main__":
#  result=predict_incident_severity_by_tf(None)
#  print(result)
